# Crawledin - Parte 1:

Após instalar as bibliotecas necessárias, siga executando uma célula por vez, após anterior finalizar

In [1]:
import asyncio
from pyppeteer import launch
import pandas as pd
import requests
from json import loads
import time
import random
from datetime import datetime
from tqdm.notebook import tqdm


# !pip install tqdm

# Inicialização do browser para o usuário logar

Faça login até chegar na página inicial do seu linkedin


In [2]:
browser = await launch(
        headless=False,
        args=['--no-sandbox'],
        autoClose=False
    )
page = await browser.newPage()
await page.goto("https://www.linkedin.com/")

# digite se login e senha no browser e siga até estar na sua página inicial

# Vai até a página de seus seguidores para pegar headers da API

In [3]:
await page.goto("https://www.linkedin.com/feed/followers/")
global headers
headers = {}
await page.setRequestInterception(True)
async def intercept(request):
    if ("https://www.linkedin.com/voyager/api/feed/richRecommendedEntities" in request.url ):
        global headers
        headers = request.headers
        print(len(headers))
    await request.continue_()
page.on('request', lambda req: asyncio.ensure_future(intercept(req)))
await page.evaluate( "() => { window.scrollBy(0, window.innerHeight); }");
time.sleep(5)
cookie1 = await page.cookies()
s = []
for e in cookie1:
    s.append(f"{e['name']}={e['value']}")
cookie1_str = "; ".join(s)
headers = {key: str(value) for key, value in headers.items()}
headers["cookie"] = cookie1_str

8


# Role um pouco o browser para carregar mais seguidores.
### Com isso poderemos pegar o cabeçalho da requisição e utilizar uma forma mais performática para pegar as informações sem utilizar o browser

In [4]:
# await page.evaluate( "() => { window.scrollBy(0, window.innerHeight); }");
time.sleep(5)
cookie1 = await page.cookies()
s = []
global headers
for e in cookie1:
    s.append(f"{e['name']}={e['value']}")
cookie1_str = "; ".join(s)
print(headers)
headers = {key: str(value) for key, value in headers.items()}
headers["cookie"] = cookie1_str
countw = 20
index_start = 280
url_inf_scroll = "https://www.linkedin.com/voyager/api/feed/richRecommendedEntities?count={countw}&q=followerRecommendations&start={index_start}"


{'x-li-track': '{"clientVersion":"1.7.6692","mpVersion":"1.7.6692","osName":"web","timezoneOffset":-2,"deviceFormFactor":"DESKTOP","mpName":"voyager-web","displayDensity":1,"displayWidth":1366,"displayHeight":768}', 'x-li-lang': 'pt_BR', 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3542.0 Safari/537.36', 'x-li-page-instance': 'urn:li:page:d_flagship3_followers_hub;oy9LMKt3TA63zonGEG3BFw==', 'accept': 'application/vnd.linkedin.normalized+json+2.1', 'csrf-token': 'ajax:8321149921634352893', 'x-restli-protocol-version': '2.0.0', 'referer': 'https://www.linkedin.com/feed/followers/'}


# Funções para puxar dados do LinkedIn

In [9]:
def get_data(countw, index_start, url, headers):
    try:
        resp = requests.get(url=url.format(countw=countw, index_start=index_start), headers=headers)
        if resp.status_code == 200:
            data = loads(resp.content.decode("utf-8"))
            return data
        else:
            raise Exception(f"not allowed. headers or coockies problems. Status: {resp.status_code}")
    except Exception as e:
        print(f"error: {e}")
        return []
    
def agrupa_entidades(elements, data):
    for element in data['data']['elements']:
        for inc in data['included']:
            ek = element.get("*recommendedEntity", "a").split(":member:")[-1]
            ik = inc.get('entityUrn',"b").split(":")[-1]
            if ek == ik:
                e = dict()
                e.update(element)
                e.update(inc)
                elements[ek] = elements.get(ek, {})
                elements[ek].update(e)
    return elements

def dict_element(e):
    return {
        "seguidores": e["followerCount"],
        "influenciador": e["influencer"],
        "nome": f"{e['firstName']} {e['lastName']}",
        "cargo": e["occupation"],
        "link": f'https://www.linkedin.com/in/{e["publicIdentifier"]}',
        "prova_social": e["socialProofTotalCount"],
        "publicacoes": f'https://www.linkedin.com/in/{e["publicIdentifier"]}/detail/recent-activity/shares/',
        "localizacao": e["locationName"]
    }

def dataFrame_element(e):
    return pd.DataFrame.from_dict([dict_element(e)])

                                  
def get_data_in_range(groups_qty, countw, first_index_start, url_inf_scroll, headers, wait_time, wait_random_time):
    df = pd.DataFrame([], columns=["seguidores", "influenciador", "nome", "cargo", "link", "prova_social", "publicacoes", "localizacao"])
    for i in tqdm(range(groups_qty)):
        data1 = get_data(countw, 1000 + countw*i, url_inf_scroll, headers)
        elements1 = dict()
        agrupa_entidades(elements1, data1)
        time.sleep(wait_time + wait_random_time * random.random())
        for e in elements1.values():
            df_t = dataFrame_element(e)
            df = df.append(df_t, ignore_index=True)
    return df
            

# Essa etapa executa de fato a consulta

No primeiro parâmetro você passa seu número de seguidores divido por 20. Pode passar menos também, para trazer apenas uma amostra.
Os dois ultimos parâmetros são de tempo para esperar entre uma requisição. Se quiser zerar ele fica mais rápido, mas não recomendo.

In [10]:
countw = 20
index_start = 280
url_inf_scroll = "https://www.linkedin.com/voyager/api/feed/richRecommendedEntities?count={countw}&q=followerRecommendations&start={index_start}"

df = get_data_in_range(124, countw, 0, url_inf_scroll, headers, .1, .5)
df

,seguidores,influenciador,nome,cargo,link,prova_social,publicacoes,localizacao
0,24268,False,Jairo Andrade Junior,"Engenheiro, Gestor e Consultor",https://www.linkedin.com/in/jairo-andrade-juni...,1,https://www.linkedin.com/in/jairo-andrade-juni...,None
1,1123,False,Ana Carolina Leo,Executive and Legal Director | Norte Global Tr...,https://www.linkedin.com/in/ana-carolina-leo-8...,0,https://www.linkedin.com/in/ana-carolina-leo-8...,None
2,475,False,Cazuza Neto,Software Engineer na Jaya Tech,https://www.linkedin.com/in/cazuzaneto,0,https://www.linkedin.com/in/cazuzaneto/detail/...,None
3,4484,False,Fernanda Camargos,Psicóloga | RH ágil | People Experience | Dese...,https://www.linkedin.com/in/fernanda-camargos-...,1,https://www.linkedin.com/in/fernanda-camargos-...,None
4,2772,False,Raphael Henrique,iOS Developer | Front-End,https://www.linkedin.com/in/raphael-henrique-b...,2,https://www.linkedin.com/in/raphael-henrique-b...,None
...,...,...,...,...,...,...,...,...
1521,15365,False,Ana Carolina Buschinelli,Tech Recruiter | Talent Acquisition | Mentora ...,https://www.linkedin.com/in/ana-carolina-busch...,5,https://www.linkedin.com/in/ana-carolina-busch...,None
1522,4025,False,Lais Guedes,Desenvolvedora Front-end,https://www.linkedin.com/in/lais-guedes-692476115,0,https://www.linkedin.com/in/lais-guedes-692476...,None
1523,5556,False,Thiago Ribeiro,"Gerente Geral na ibis, ibis Styles, ibis budget",https://www.linkedin.com/in/thiago-ribeiro-75b...,0,https://www.linkedin.com/in/thiago-ribeiro-75b...,None
1524,580,False,Jose Adonai Arruda dos Santos,"Systems Engineering - UFMG, Brazil | Computer ...",https://www.linkedin.com/in/adonai-arruda,0,https://www.linkedin.com/in/adonai-arruda/deta...,None


#  Exibe o data frame ordenado por seguidores

In [11]:
df.sort_values(by="seguidores", ascending=False).head()


,seguidores,influenciador,nome,cargo,link,prova_social,publicacoes,localizacao
1490,575851,True,Gustavo Caetano,"CEO da SambaTech, Linkedin InFluencer (570k se...",https://www.linkedin.com/in/gustavocaetano,457,https://www.linkedin.com/in/gustavocaetano/det...,None
381,73722,False,Dalva Corrêa,LinkedIn Top Voices 2020 | Professora e Mentor...,https://www.linkedin.com/in/dalvacorrea,41,https://www.linkedin.com/in/dalvacorrea/detail...,None
1121,66639,False,Eduardo Felix,Especialista em Recursos Humanos e Carreiras |...,https://www.linkedin.com/in/eduardomfelix,62,https://www.linkedin.com/in/eduardomfelix/deta...,None
1491,57404,False,Débora Alcantara,Comunicadora | Vencedora do Prêmio ABRADI Prof...,https://www.linkedin.com/in/deboraalcantara,66,https://www.linkedin.com/in/deboraalcantara/de...,None
912,36998,False,Thiago Lima,Founder & CEO at LinkApi,https://www.linkedin.com/in/thiagolimabr,43,https://www.linkedin.com/in/thiagolimabr/detai...,None


# Salva o dataframe em csv com a data e hora

In [34]:
df.sort_values(by="seguidores", ascending=False).to_csv(f'crawledin-{datetime.now().strftime("%Y-%m-%d_%H-%M")}.csv', index=False)

# Visita a página de seguidores com conteudo.

In [54]:
for index, row in df.sample(20).iterrows():
    print(row['link'])
    await page.goto(row['publicacoes'])
    elements = (await page.querySelectorAll("h1"))
    tem_post = True
    for e in elements:
        if "Nada para ver por enquanto" in await page.evaluate('(element) => element.textContent', e):
            tem_post = False
    if tem_post:
        time.sleep(9 + 2 * random.random())
    else:
        time.sleep(1)



https://www.linkedin.com/in/donizete-crisostomo-4b24a318b
https://www.linkedin.com/in/thais-matos-acacio
https://www.linkedin.com/in/phelipe-salustiano


Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error (Target.sendMessageToTarget): No session with given id')>
pyppeteer.errors.NetworkError: Protocol error (Target.sendMessageToTarget): No session with given id


https://www.linkedin.com/in/carolvieirav


Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error (Target.sendMessageToTarget): No session with given id')>
pyppeteer.errors.NetworkError: Protocol error (Target.sendMessageToTarget): No session with given id


https://www.linkedin.com/in/maycon-jean-de-moura-25273a158


...

Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error (Target.sendMessageToTarget): No session with given id')>
pyppeteer.errors.NetworkError: Protocol error (Target.sendMessageToTarget): No session with given id


https://www.linkedin.com/in/larissa-bicalho-pimenta-05a597119


Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error (Target.sendMessageToTarget): No session with given id')>
pyppeteer.errors.NetworkError: Protocol error (Target.sendMessageToTarget): No session with given id


https://www.linkedin.com/in/rogeriocs


Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error (Target.sendMessageToTarget): No session with given id')>
pyppeteer.errors.NetworkError: Protocol error (Target.sendMessageToTarget): No session with given id
